In [12]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [13]:
var('xx')
Q.<i> = NumberField(xx^2 + 1)

In [14]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [15]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 7.93 s, sys: 288 ms, total: 8.22 s
Wall time: 19.2 s


576

In [16]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 4.27 s, sys: 248 ms, total: 4.52 s
Wall time: 26.1 s


2

In [22]:
with open('all_L_sets.pickle', 'rb') as fil:
    all_L_sets = pickle.load(fil)

In [23]:
possible_L_sets = [L_set for L_set in all_L_sets if L_set[0:3] == ('E1', 'G4', 'E3')]
possible_projs = SE1.find_all_proj_parallel(possible_L_sets)

In [47]:
%time SE1.find_conditions_for_subfamilies(possible_projs)

CPU times: user 9.52 s, sys: 35.2 ms, total: 9.55 s
Wall time: 9.59 s


[[4*c + (i)*d + (-i)*e + (i + 2)*f, 5*d^2 + (4*i - 2)*d*e + (-4*i - 3)*e^2 + (-8*i - 6)*d*f + (-36*i + 38)*e*f + (24*i - 7)*f^2],
 [4*c + (-i)*d + (i)*e + (-i + 2)*f, 5*d^2 + (-4*i - 2)*d*e + (4*i - 3)*e^2 + (8*i - 6)*d*f + (36*i + 38)*e*f + (-24*i - 7)*f^2],
 [4*c + (i)*d + (i + 2)*e + (-i)*f, 5*d^2 + (-8*i - 6)*d*e + (24*i - 7)*e^2 + (4*i - 2)*d*f + (-36*i + 38)*e*f + (-4*i - 3)*f^2],
 [4*c + (-i)*d + (-i + 2)*e + (i)*f, 5*d^2 + (8*i - 6)*d*e + (-24*i - 7)*e^2 + (-4*i - 2)*d*f + (36*i + 38)*e*f + (4*i - 3)*f^2]]

In [27]:
%%time
mon = ((x+y+z+t)^3).monomials()
primary_decs = []
ideals = []
for proj in possible_projs:
    nc = remove_sing_factors(SE1.eqn.subs(change_coord(proj)), SE1.sing_locus)
    conds = list(set(matrix([[SE1.coefficient(mn) for mn in mon], [nc.coefficient(mn) for mn in mon]]).minors(2)))
    red_conds = [remove_sing_factors(cond, SE1.sing_locus) for cond in conds if cond !=0]
    ideal = P.ideal(red_conds)
    primary_dec = [ideal for ideal in ideal.radical().primary_decomposition() if SE1.is_ideal_valid(ideal)]
    ideals += primary_dec
    primary_decs.append(primary_dec)

CPU times: user 9.04 s, sys: 19.2 ms, total: 9.06 s
Wall time: 9.11 s


In [28]:
for ii in range(len(possible_L_sets)):
    if primary_decs[ii] != []:
        print(possible_L_sets[ii])
        print(primary_decs[ii])

('E1', 'G4', 'E3', 'G6', 'F24')
[Ideal (4*c + (i)*d + (-i)*e + (i + 2)*f, 5*d^2 + (4*i - 2)*d*e + (-4*i - 3)*e^2 + (-8*i - 6)*d*f + (-36*i + 38)*e*f + (24*i - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1, Ideal (4*c + (-i)*d + (i)*e + (-i + 2)*f, 5*d^2 + (-4*i - 2)*d*e + (4*i - 3)*e^2 + (8*i - 6)*d*f + (36*i + 38)*e*f + (-24*i - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1]
('E1', 'G4', 'E3', 'G5', 'F24')
[Ideal (4*c + (i)*d + (i + 2)*e + (-i)*f, 5*d^2 + (-8*i - 6)*d*e + (24*i - 7)*e^2 + (4*i - 2)*d*f + (-36*i + 38)*e*f + (-4*i - 3)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1, Ideal (4*c + (-i)*d + (-i + 2)*e + (i)*f, 5*d^2 + (8*i - 6)*d*e + (-24*i - 7)*e^2 + (-4*i - 2)*d*f + (36*i + 38)*e*f + (4*i - 3)*f^2) of Multivariate Polynom

In [36]:
ideale = ideals[0]

In [40]:
poly_e = ideale.gens()[0]
e_sost = {e: -(poly_e-poly_e.coefficient(e)*e)/poly_e.coefficient(e)}

In [41]:
poly_d = [gen for gen in ideale.subs(e_sost).gens() if gen !=0][0]
d_sost = {d:-(poly_d-poly_d.coefficient(d)*d)/poly_d.coefficient(d)}

In [42]:
[gen.subs(e_sost).subs(d_sost) for gen in ideale.gens()]

[0, 0]

In [43]:
SE1_1 = SE1.subs(e_sost).subs(d_sost)

In [45]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 2.47 s, sys: 218 ms, total: 2.69 s
Wall time: 21.6 s


576

In [46]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 1.62 s, sys: 182 ms, total: 1.81 s
Wall time: 8.35 s


4

In [47]:
lines_perms = []
for simm in simm_SE1_1:
    lines_perms.append(Permutation(SE1_1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(4, 'C4')

In [48]:
lines_perms_group.gens_small()

[(2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20)]

In [49]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46')]